In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.metrics import plot_confusion_matrix

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

from sklearn.ensemble import RandomForestClassifier
from time import time


folder = "../Collected_Data/FormattedEEG/"
types = ["control", "left_blink", "right_blink"]
paths = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])

# Read in csv files
def read_csvs(folder, types, paths):
    output_x = []
    output_y = []
    
    for path in paths:
        for i in range(1, 11):
            if path[0].find("control") != -1 and i == 6:
                break;
            cur_path = folder + types[int(path[1])] + "/" + path[0] + "_" + str(i) + "_" + "formatted.csv"
            input_data = pd.read_csv(cur_path, header = 0, usecols=[1, 2, 3, 4])
            X = input_data.to_numpy()[250:1000]
            X = X.T
            y = path[1]
            output_x.append(X)
            output_y.append(float(y))
    
    output_x = np.array(output_x) 
    output_y = np.array(output_y)
    return output_x, output_y

X, y = read_csvs(folder, types, paths)


cv = KFold(n_splits=4, shuffle=True)

start = time()
clf = make_pipeline(XdawnCovariances(estimator='oas'), TangentSpace(metric = "riemann"), RandomForestClassifier(n_estimators=1000, n_jobs = 8))

pred = np.zeros(len(y))
print(len(y))

for train_idx, test_idx in cv.split(X):
    y_train, y_test = y[train_idx], y[test_idx]
    clf.fit(X[train_idx], y_train)
    pred[test_idx] = clf.predict(X[test_idx])

acc = np.mean(pred == y)
print("Classification accuracy: %f " % (acc))
end = time()
print("Time taken: " + str(end - start))
plot_confusion_matrix(clf, X, y, display_labels=types)  
plt.show()  

In [ ]:
from time import time

folder = "../Collected_Data/FormattedEEG/"
types = ["control", "left_blink", "right_blink"]
paths_orig = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
noleftvar1 = np.array([["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
noleftvar2 = np.array([["left_blink_var1", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
norightvar1 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var2", 2], ["control_var1", 0], ["control_var2", 0]])
norightvar2 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["control_var1", 0], ["control_var2", 0]])
nocontrol1 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var2", 0]])
nocontrol2 = np.array([["left_blink_var1", 1],["left_blink_var2", 1], ["right_blink_var1", 2], ["right_blink_var2", 2], ["control_var1", 0]])

paths_list = [[paths_orig, "original"], [noleftvar1, "no left blink var1"], [noleftvar2, "no left blink var2"], [norightvar1, "no right var1"], [norightvar2, "no right var2"], [nocontrol1, "no control var 1"], [nocontrol2, "no control var 2"]]
for paths in paths_list:
    X, y = read_csvs(folder, types, paths[0])


    cv = KFold(n_splits=4, shuffle=True)

    clf = make_pipeline(XdawnCovariances(estimator='oas'), TangentSpace(metric = "riemann"), RandomForestClassifier(n_estimators=500, n_jobs=4))

    pred = np.zeros(len(y))
    print("Test: " + paths[1])
    print("Test size:" + str(len(y)))

    for train_idx, test_idx in cv.split(X):
        y_train, y_test = y[train_idx], y[test_idx]
        clf.fit(X[train_idx], y_train)
        pred[test_idx] = clf.predict(X[test_idx])

    count = 0
    wrong_labelled = []
    for j in range(len(pred)):
        if pred[j] == y[j]:
            count += 1
        else:
            wrong_labelled.append(j)
    print(wrong_labelled)
    acc = count / len(pred)
    print("Classification accuracy: %f \n" % (acc))
    plot_confusion_matrix(clf, X, y, display_labels=types)  
    plt.show()  

looking for an EEG stream...
[ 4.59472656e+02  4.71191406e+02 -9.17968750e+02  2.54394531e+02
  9.70561438e+05 -1.34309998e-03]
[-1.00000000e+03 -1.00000000e+03 -7.42187500e+01 -1.00000000e+03
  9.70561438e+05  2.56315013e-03]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70561438e+05  6.46940013e-03]
[ 3.21289062e+02  2.67578125e+02 -1.87988281e+02  5.32226562e+02
  9.70561438e+05  1.03756497e-02]
[ 8.52050781e+02  8.99902344e+02 -9.84863281e+02  5.54199219e+02
  9.70561438e+05  1.42818997e-02]
[-7.54882812e+02 -6.73339844e+02 -1.30859375e+02 -9.75097656e+02
  9.70561438e+05  1.81881506e-02]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70561438e+05  2.20944006e-02]
[-1.95800781e+02 -1.99707031e+02 -5.03417969e+02 -4.83398438e+01
  9.70561438e+05  2.60006506e-02]
[-8.63281250e+02 -8.54492188e+02 -9.92187500e+02 -9.87792969e+02
  9.70561438e+05  2.99069006e-02]
[ 5.27343750e+01  9.37500000e+01  3.62304688e+02 -3.93554688e+02
  9.70561500e+0

[-8.49609375e+02 -8.66699219e+02  6.30859375e+02 -8.50585938e+02
  9.70562250e+05 -5.17790020e-03]
[ 2.45117188e+02  2.50976562e+02  7.22656250e+02  1.45996094e+02
  9.70562250e+05 -1.27164996e-03]
[-1.00000000e+03 -1.00000000e+03 -7.14843750e+02 -1.00000000e+03
  9.70562250e+05  2.63460004e-03]
[-8.37890625e+02 -8.63769531e+02 -1.00000000e+03 -6.22558594e+02
  9.70562250e+05  6.54084980e-03]
[ 7.86132812e+02  7.52929688e+02  2.14843750e+01 -9.87304688e+02
  9.70562250e+05  1.04470998e-02]
[ 6.52832031e+02  6.43066406e+02 -8.93066406e+02  2.64648438e+02
  9.70562250e+05  1.43533498e-02]
[-9.14062500e+02 -9.24804688e+02 -5.22460938e+01 -1.00000000e+03
  9.70562250e+05  1.82595998e-02]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70562250e+05  2.21658498e-02]
[ 2.49023438e+01  4.10156250e+01 -2.48046875e+02  3.18847656e+02
  9.70562250e+05  2.60720998e-02]
[-8.57910156e+02 -8.84765625e+02 -8.73535156e+02  6.96777344e+02
  9.70562250e+05  2.99783498e-02]
[-2.001953

[-9.40917969e+02 -9.48730469e+02 -8.76953125e+02  6.05957031e+02
  9.70562688e+05  2.36831997e-02]
[-3.93066406e+02 -3.95996094e+02  7.86132812e+01 -9.39941406e+02
  9.70562688e+05  2.75894497e-02]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70562750e+05 -3.10043003e-02]
[-2.06054688e+02 -2.09960938e+02 -5.81542969e+02 -1.68457031e+02
  9.70562750e+05 -2.70980503e-02]
[-8.43750000e+02 -8.39843750e+02  9.64355469e+02 -9.50195312e+02
  9.70562750e+05 -2.31918003e-02]
[ 1.42578125e+02  1.91406250e+02  3.92089844e+02 -2.61230469e+02
  9.70562750e+05 -1.92855503e-02]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70562750e+05 -1.53793003e-02]
[-4.03320312e+02 -4.14550781e+02 -8.87695312e+02 -2.09472656e+02
  9.70562750e+05 -1.14730503e-02]
[-8.74023438e+02 -8.71582031e+02  6.45507812e+02 -8.40332031e+02
  9.70562750e+05 -7.56679988e-03]
[ 2.49023438e+02  2.48535156e+02  6.78222656e+02  2.32910156e+02
  9.70562750e+05 -3.66054988e-03]
[-1.000000

[-1.00000000e+03 -1.00000000e+03 -8.49609375e+01 -1.00000000e+03
  9.70563125e+05  6.37730025e-03]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70563125e+05  1.02835502e-02]
[ 2.84179688e+02  2.24609375e+02 -2.31933594e+02  5.31738281e+02
  9.70563125e+05  1.41898002e-02]
[ 9.56542969e+02  9.99023438e+02 -8.78906250e+02  5.60546875e+02
  9.70563125e+05  1.80960502e-02]
[-5.74218750e+02 -4.99511719e+02  5.51757812e+01 -9.71679688e+02
  9.70563125e+05  2.20023002e-02]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70563125e+05  2.59085502e-02]
[-1.99218750e+02 -2.01660156e+02 -4.88769531e+02 -4.88281250e+01
  9.70563125e+05  2.98148002e-02]
[-8.55468750e+02 -8.48632812e+02 -9.76562500e+02 -9.85351562e+02
  9.70563188e+05 -2.87789498e-02]
[ 6.98242188e+01  9.66796875e+01  3.60839844e+02 -3.89648438e+02
  9.70563188e+05 -2.48726998e-02]
[-1.00000000e+03 -1.00000000e+03 -1.00000000e+03 -1.00000000e+03
  9.70563188e+05 -2.09664498e-02]
[-3.359375